<a href="https://colab.research.google.com/github/AjithNJIT/Data_Mining_Final/blob/main/Data_Mining_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

                **CS634- DATA MINING -AC2449**

                **SEMANTIC SEARCH ANALYSIS USING SBERT**

**This Project covers the Semantic Search using SBERT on Youtube videos dataset. It **covers** the Pytorch implementation of nearest neighbours search. We have used the Approximate Knn with the help of an multilingual embeddings model that was **used** for quora **data** **analysis**.

**We** **have** used **Fox** **News** **channel** Id to Pull all **the** video links. The **Video** **Id** **link** that was retrived is **massaged** to **fetch** the video id from the entire link

Example :
https://www.youtube.com/watch?v=UyXJva9vg9o
After Massaging:
UyXJva9vg9o 

In [ ]:

import scrapetube
import sys

path = '_list.txt'

print('**********************\n')
print("The result will be saved in '_list.txt' file.")
print("Enter Channel ID:")

# Prints the output in the console and into the '_list.txt' file.
class Logger:
 
    def __init__(self, filename):
        self.console = sys.stdout
        self.file = open(filename, 'w')
 
    def write(self, message):
        self.console.write(message)
        self.file.write(message)
 
    def flush(self):
        self.console.flush()
        self.file.flush()

sys.stdout = Logger(path)

# Strip the: "https://www.youtube.com/channel/"
channel_id_input = input()
channel_id = channel_id_input.strip("https://www.youtube.com/channel/")

videos = scrapetube.get_channel(channel_id)

for video in videos:
    print("https://www.youtube.com/watch?v="+str(video['videoId']))
#    print(video['videoId'])

**Once the Data(Video) is Fetched , then We have run a api to retrive the Title and youtube link inorder to feed it to the SBERT Model to create Embeddings**


In [ ]:
from typing import Iterable, Tuple, Dict, List, Any

from youtube_transcript_api import YouTubeTranscriptApi

import urllib.request
import json
import urllib


def fetch_transcript(vid_id) -> Tuple[str, List[Dict[str, Any]]]:
    params = {"format": "json", "url": f"https://www.youtube.com/watch?v={vid_id}"}
    #print(params)
    url = "https://www.youtube.com/oembed"
    #print(f"Video id {vid_id}")

    query_string = urllib.parse.urlencode(params)
    url = url + "?" + query_string

    #print(f"Fetching from {url}")

    with urllib.request.urlopen(url) as response:
        response_text = response.read()
        title = json.loads(response_text.decode())["title"]

    # retrieve the available transcripts
    transcript_list = YouTubeTranscriptApi.list_transcripts(vid_id)
    
    utube_url = f"https://www.youtube.com/watch?v={vid_id}"
    #print(utube_url)
    
    #return vid_id,title, transcript_list.find_transcript(['en']).fetch()
    return vid_id,title,url


def fetch_transcript_gen(vid_ids: Iterable):
    for video_id in vid_ids:
        yield fetch_transcript(video_id)


if __name__ == '__main__':
    from pprint import pprint
#    for vid_id,title_, data_ in fetch_transcript_gen(['UyXJva9vg9o','6ZZONMmn3os']):
videoListName = "/home/ubuntu/you_tube_video_ids.txt"
with open(videoListName) as f:
    video_ids = f.read().splitlines()
    for vid_id,title_,url in fetch_transcript_gen(video_ids):
        #print(f"vid_id: {vid_id}\tTitle: {title_}\t Data length: {len(data_)}\n")
        #print(f"\tTitle: {title_}\turl:https://www.youtube.com/watch?v={vid_id}")
        print(f"{vid_id}\t{title_}|https://www.youtube.com/watch?v={vid_id}")
        #pprint(data_)




**Once We have the Video Id and the Title with youtube Link, we are ready to call the Pre-trained Model to Embed the data and then run queires to get the results performing Knn. Below is an example**

In [ ]:
from sentence_transformers import SentenceTransformer, util
import os
import csv
import pickle
import time
import hnswlib

##IMPORTING THE MODEL TO EMBED OUR DATA####

model_name = 'quora-distilbert-multilingual'
model = SentenceTransformer(model_name)

dataset_path="/home/ubuntu/youtube_video_list.tsv"

##As the Dataset is Small I have added 93 as the Corpus Size
max_corpus_size = 93

embedding_cache_path = 'youtube-embeddings-{}-size-{}.pkl'.format(model_name.replace('/', '_'), max_corpus_size)


embedding_size = 100    #Size of embeddings
top_k_hits = 3         #Pulling only First 3 hits

#Check if embedding cache path exists
if not os.path.exists(embedding_cache_path):
    # Check if the dataset exists. If not, download and extract
    # Get all unique sentences from the file
    corpus_sentences = set()
    
    with open(dataset_path, encoding='utf8') as fIn:
        reader = csv.DictReader(fIn, delimiter='\t', quoting=csv.QUOTE_MINIMAL)
        for row in reader:
            #print(row)
            corpus_sentences.add(row['Youtube_title'])
            if len(corpus_sentences) >= max_corpus_size:
                break

    corpus_sentences = list(corpus_sentences)
    print("Encode the corpus. This might take a while")
    ###Creating EMbeddings and storing the file on local disc
    corpus_embeddings = model.encode(corpus_sentences, show_progress_bar=True, convert_to_numpy=True)
    print("Store file on disc")
    with open(embedding_cache_path, "wb") as fOut:
        pickle.dump({'sentences': corpus_sentences, 'embeddings': corpus_embeddings}, fOut)
else:
    print("Load pre-computed embeddings from disc")
    with open(embedding_cache_path, "rb") as fIn:
        cache_data = pickle.load(fIn)
        corpus_sentences = cache_data['sentences']
        corpus_embeddings = cache_data['embeddings']

#Defining our hnswlib index
index_path = "./hnswlib.index"
#We use Inner Product (dot-product) as Index. We will normalize our vectors to unit length, then is Inner Product equal to cosine similarity
index = hnswlib.Index(space = 'cosine', dim = embedding_size)

if os.path.exists(index_path):
    print("Loading index...")
    index.load_index(index_path)
else:
    ### Create the HNSWLIB index
    print("Start creating HNSWLIB index")
    index.init_index(max_elements = len(corpus_embeddings), ef_construction = 400, M = 64)

    # Then we train the index to find a suitable clustering
    index.add_items(corpus_embeddings, list(range(len(corpus_embeddings))))

    print("Saving index to:", index_path)
    index.save_index(index_path)

# Controlling the recall by setting ef:
index.set_ef(50)  # ef should always be > top_k_hits

######### Search in the index ###########

print("Corpus loaded with {} sentences / embeddings".format(len(corpus_sentences)))

while True:
    inp_question = input("Please enter a question: ")

    start_time = time.time()
    question_embedding = model.encode(inp_question)

    #We use hnswlib knn_query method to find the top_k_hits
    corpus_ids, distances = index.knn_query(question_embedding, k=top_k_hits)

    # We extract corpus ids and scores for the first query
    hits = [{'corpus_id': id, 'score': 1-score} for id, score in zip(corpus_ids[0], distances[0])]
    hits = sorted(hits, key=lambda x: x['score'], reverse=True)
    end_time = time.time()

    print("Input question:", inp_question)
    print("Results (after {:.3f} seconds):".format(end_time-start_time))

    # Approximate Nearest Neighbor (ANN) search
    correct_hits = util.semantic_search(question_embedding, corpus_embeddings, top_k=top_k_hits)[0]
    correct_hits_ids = set([hit['corpus_id'] for hit in correct_hits])
    
    print("correct hits", correct_hits)
    print("correct_hits_ids", correct_hits_ids)

    knn_corpus_ids = set([hit['corpus_id'] for hit in hits])
        
    if len(knn_corpus_ids) != len(correct_hits_ids):
        print("Approximate Nearest Neighbor returned a different number of results than expected")

    recall = len(knn_corpus_ids.intersection(correct_hits_ids)) / len(correct_hits_ids)
    
    print("Results:")
    for hit in correct_hits[0:top_k_hits]:
        if hit['corpus_id'] not in knn_corpus_ids:
            print("\t{:.3f}\t{}".format(hit['score'], corpus_sentences[hit['corpus_id']]))
    print("\n\n========\n")



Encode the corpus. This might take a while
Batches: 100%
3/3 [00:06<00:00, 1.94s/it]
Store file on disc
Loading index...
Corpus loaded with 93 sentences / embeddings
Please enter a question: how is Biden doing in his presidency
Input question: how is Biden doing in his presidency
Results (after 0.518 seconds):
correct hits [{'corpus_id': 5, 'score': 0.8723465204238892}, {'corpus_id': 69, 'score': 0.8607958555221558}, {'corpus_id': 78, 'score': 0.8515018224716187}]
correct_hits_ids {69, 5, 78}
Results:
	0.872	Biden adviser defends the president’s economy|https://www.youtube.com/watch?v=mKYi3TvX1U8
	0.861	What's behind the Biden family's 'opulent' lifestyle?|https://www.youtube.com/watch?v=3OKIvDDNAC8
	0.852	Perino: You have to wonder who the White House spin on 'recession' is for|https://www.youtube.com/watch?v=FrPEzSRhmEs


========

Please enter a question: Who are all in Senate
Input question: Who are all in Senate
Results (after 0.344 seconds):
correct hits [{'corpus_id': 87, 'score': 0.8043516278266907}, {'corpus_id': 74, 'score': 0.780350923538208}, {'corpus_id': 79, 'score': 0.767914891242981}]
correct_hits_ids {74, 79, 87}
Results:
	0.804	House Minority Leader McCarthy holds press conference|https://www.youtube.com/watch?v=PczZ6mI7390
	0.780	Karine Jean-Pierre holds a White House briefing | 7/29/22|https://www.youtube.com/watch?v=-vI5kRdne1Q
	0.768	Biden admin. puts politics above the American public: National Border Patrol Council VP|https://www.youtube.com/watch?v=2OxlSY0Mx40


========

Please enter a question: Which Party does Biden Belong to
Input question: Which Party does Biden Belong to
Results (after 0.333 seconds):
correct hits [{'corpus_id': 69, 'score': 0.8159942626953125}, {'corpus_id': 71, 'score': 0.8026124835014343}, {'corpus_id': 16, 'score': 0.7910526394844055}]
correct_hits_ids {16, 69, 71}
Results:
	0.816	What's behind the Biden family's 'opulent' lifestyle?|https://www.youtube.com/watch?v=3OKIvDDNAC8
	0.803	Biden family 'too big to touch'|https://www.youtube.com/watch?v=_cpIbRimgOI
	0.791	'The Five' on the media claiming Biden is 'back in the game'|https://www.youtube.com/watch?v=4cx4BJSm0Zc


========

Please enter a question: How Far is china?
Input question: How Far is china?
Results (after 0.323 seconds):
correct hits [{'corpus_id': 41, 'score': 0.8983137011528015}, {'corpus_id': 27, 'score': 0.8780466318130493}, {'corpus_id': 15, 'score': 0.8729186654090881}]
correct_hits_ids {41, 27, 15}
Results:
	0.898	Tensions with China escalate as Pelosi visits Asia|https://www.youtube.com/watch?v=NsCb1-_a8_E
	0.878	Biden threatened by China over Pelosi's Taiwan plans|https://www.youtube.com/watch?v=8wQOOZVTJ7o
	0.873	China issued an 'unacceptable threat' on Pelosi's trip, Biden once again not being clear: Waltz|https://www.youtube.com/watch?v=w8Hr91l7AM0


========